# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import requests
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [33]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))
print(cities)

597
['souillac', 'mrirt', 'cape town', 'alofi', 'luena', 'mahadday weyne', 'butaritari', 'ngukurr', 'tsihombe', 'kununurra', 'hermanus', 'vaini', 'brae', 'port alfred', 'taolanaro', 'cabo san lucas', 'vanavara', 'bluff', 'barrow', 'chabahar', 'mataura', 'albany', 'kodiak', 'kapaa', 'watsa', 'nizhneyansk', 'georgetown', 'isiro', 'atuona', 'ponta do sol', 'hobart', 'cayenne', 'pisco', 'filadelfia', 'bredasdorp', 'illoqqortoormiut', 'hualmay', 'chibombo', 'springbok', 'mahebourg', 'saint-georges', 'vao', 'canton', 'sentyabrskiy', 'ushuaia', 'tiksi', 'punta arenas', 'luderitz', 'union', 'songkhla', 'castro', 'leshukonskoye', 'cidreira', 'east london', 'mount isa', 'bokspits', 'malmyzh', 'lorengau', 'chuy', 'rikitea', 'yellowknife', 'alice springs', 'komsomolskiy', 'dikson', 'busselton', 'hami', 'lagoa', 'flin flon', 'taiyuan', 'sterling', 'chapais', 'polyarnyy', 'bambous virieux', 'hithadhoo', 'dudinka', 'shar', 'port lincoln', 'pevek', 'narsaq', 'terme', 'kensington', 'bethel', 'qaanaaq',

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [35]:
#for city in cities:
weather_url = f"https://api.openweathermap.org/data/2.5/weather?q={cities[0]}&appid={weather_api_key}"
print(weather_url)
response = requests.get(weather_url).json()
print(response["weather"][0]["main"])
print(response["main"]["temp"])

https://api.openweathermap.org/data/2.5/weather?q=souillac&appid=9ce71775c97a7b35fb443ccc88625786
Rain
297.15


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression